In [1]:
#https://raw.githubusercontent.com/rishabhmisra/News-Headlines-Dataset-For-Sarcasm-Detection/master/Sarcasm_Headlines_Dataset.json

In [2]:
!python --version

Python 3.7.7


In [3]:
# # Sarcasm Detection **Acknowledgement**

# Misra, Rishabh, and Prahal Arora. "Sarcasm Detection using Hybrid Neural Network." arXiv preprint arXiv:1908.07414 (2019).

# **Required Files given in below link.**

# https://drive.google.com/drive/folders/1xUnF35naPGU63xwRDVGc-DkZ3M8V5mMk

## Install `Tensorflow2.0` 

In [ ]:
# !!pip uninstall tensorflow
# !pip install tensorflow==2.0.0

## Get Required Files from Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [ ]:
#Set your project path 
# project_path =  ## Add your path here ##

#**## Reading and Exploring Data**

## Read Data "Sarcasm_Headlines_Dataset.json". Explore the data and get  some insights about the data. ( 4 marks)
Hint - As its in json format you need to use pandas.read_json function. Give paraemeter lines = True.

In [4]:
import pandas as pd

In [5]:
headlines_df = pd.read_json("Sarcasm_Headlines_Dataset.json",lines=True)

In [6]:
headlines_df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [7]:
headlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
 2   article_link  28619 non-null  object
dtypes: int64(1), object(2)
memory usage: 670.9+ KB


## Drop `article_link` from dataset. ( 2 marks)
As we only need headline text data and is_sarcastic column for this project. We can drop artical link column here.

In [8]:
headlines_df.drop(['article_link'],axis=1,inplace=True)
headlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
dtypes: int64(1), object(1)
memory usage: 447.3+ KB


## Get the Length of each line and find the maximum length. ( 4 marks)
As different lines are of different length. We need to pad the our sequences using the max length.

In [9]:
headlines_df['length'] = headlines_df['headline'].apply(lambda headline: len(headline.split(" ")))
headlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
 2   length        28619 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 670.9+ KB


In [10]:
headlines_df.head()

,is_sarcastic,headline,length
0,1,thirtysomething scientists unveil doomsday clo...,8
1,0,dem rep. totally nails why congress is falling...,13
2,0,eat your veggies: 9 deliciously different recipes,7
3,1,inclement weather prevents liar from getting t...,8
4,1,mother comes pretty close to using word 'strea...,9


#**## Modelling**

## Import required modules required for modelling.

In [11]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

# Set Different Parameters for the model. ( 2 marks)

In [13]:
max_features = 10000
maxlen = 25
embedding_size = 200

## Apply Keras Tokenizer of headline column of your data.  ( 4 marks)
Hint - First create a tokenizer instance using Tokenizer(num_words=max_features) 
And then fit this tokenizer instance on your data column df['headline'] using .fit_on_texts()

In [17]:
t = Tokenizer(num_words=max_features)


In [18]:
t.fit_on_texts(list(headlines_df['headline']))

# Define X and y for your model.

In [20]:
X = t.texts_to_sequences(headlines_df['headline'])
X = pad_sequences(X, maxlen = maxlen)
y = np.asarray(headlines_df['is_sarcastic'])

print("Number of Samples:", len(X))
print(X[0])
print("Number of Labels: ", len(y))
print(y[0])

Number of Samples: 28619
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0  354 3166 7473 2643    2  660 1118]
Number of Labels:  28619
1


## Get the Vocabulary size ( 2 marks)
Hint : You can use tokenizer.word_index.

In [23]:
vocab_size = len(t.word_index) + 1
print("Vocabulary size is ",vocab_size)

Vocabulary size is  30885


**Word Embedding**

## Get Glove Word Embeddings

In [ ]:
# glove_file = project_path + "glove.6B.zip"

In [ ]:
# #Extract Glove embedding zip file
# from zipfile import ZipFile
# with ZipFile(glove_file, 'r') as z:
#   z.extractall()

# Get the Word Embeddings using Embedding file as given below.

In [26]:
EMBEDDING_FILE = 'glove.6B.200d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE, encoding="utf8"):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd



# Create a weight matrix for words in training docs

In [29]:
embedding_matrix = np.zeros((vocab_size, 200))

for word, i in t.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

len(embeddings.values())

400000

## Create and Compile your Model  ( 7 marks)
Hint - Use Sequential model instance and then add Embedding layer, Bidirectional(LSTM) layer, then dense and dropout layers as required. 
In the end add a final dense layer with sigmoid activation for binary classification.


In [30]:
### Embedding layer for hint 
## model.add(Embedding(num_words, embedding_size, weights = [embedding_matrix]))
### Bidirectional LSTM layer for hint 
## model.add(Bidirectional(LSTM(128, return_sequences = True)))



model = Sequential()
model.add(Embedding(vocab_size, embedding_size, weights = [embedding_matrix]))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dense(40, activation="relu"))
model.add(Dense(40, activation="relu"))
model.add(Dense(20, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

# Fit your model with a batch size of 100 and validation_split = 0.2. and state the validation accuracy ( 5 marks)


In [31]:
batch_size = 100
epochs = 5

## Add your code here ##
detectingModel = model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 22895 samples, validate on 5724 samples
Epoch 1/5
22895/22895 [==============================] - 27s 1ms/sample - loss: 0.4459 - accuracy: 0.7825 - val_loss: 0.3404 - val_accuracy: 0.8522
Epoch 2/5
22895/22895 [==============================] - 18s 801us/sample - loss: 0.2552 - accuracy: 0.8950 - val_loss: 0.3098 - val_accuracy: 0.8680
Epoch 3/5
22895/22895 [==============================] - 18s 804us/sample - loss: 0.1750 - accuracy: 0.9324 - val_loss: 0.3274 - val_accuracy: 0.8625
Epoch 4/5
22895/22895 [==============================] - 18s 805us/sample - loss: 0.1206 - accuracy: 0.9554 - val_loss: 0.3772 - val_accuracy: 0.8614
Epoch 5/5
22895/22895 [==============================] - 18s 804us/sample - loss: 0.0828 - accuracy: 0.9706 - val_loss: 0.4224 - val_accuracy: 0.8584


In [32]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, weights = [embedding_matrix]))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dense(40, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

In [33]:
batch_size = 100
epochs = 5

## Add your code here ##
detectingModel = model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 22895 samples, validate on 5724 samples
Epoch 1/5
22895/22895 [==============================] - 24s 1ms/sample - loss: 0.4688 - accuracy: 0.7807 - val_loss: 0.3319 - val_accuracy: 0.8525
Epoch 2/5
22895/22895 [==============================] - 18s 801us/sample - loss: 0.2805 - accuracy: 0.8921 - val_loss: 0.3101 - val_accuracy: 0.8663
Epoch 3/5
22895/22895 [==============================] - 18s 806us/sample - loss: 0.1956 - accuracy: 0.9298 - val_loss: 0.3433 - val_accuracy: 0.8627
Epoch 4/5
22895/22895 [==============================] - 18s 802us/sample - loss: 0.1348 - accuracy: 0.9523 - val_loss: 0.4339 - val_accuracy: 0.8625
Epoch 5/5
22895/22895 [==============================] - 18s 806us/sample - loss: 0.0949 - accuracy: 0.9674 - val_loss: 0.6805 - val_accuracy: 0.8596


In [34]:
# interestingly, adding dropout increases the val accuracy a bit more.